# Sentinel-2 example notebook

This notebook showcases how to use  `PASEOS` to simulate **Sentinel2-B (S2B)**. In particular, the notebook shows how to create `space_actors` orbiting as the **S2B** around Earth. In addition, it shows how to add a `power` device and demonstrates how to register activities to perform onboard data acquisition and processing to detect **volcanic eruptions** on `Sentinel-2 L1C data`. <br> **DISCLAIMER**: the notebook requires `rasterio` and `scikit-image` to run correctly, which is not included in the packets required to install `PASEOS`. To install `rasterio` you can use: <br><center>  ```conda install -c conda-forge rasterio``` or alternatively ```pip install rasterio``` </center>
<br>To install `scikit-image` you can use: <br><center>  ```conda install scikit-image``` or alternatively ```pip install scikit-image``` </center>

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook
import sys 
import os
sys.path.insert(1, os.path.join("..",".."))
import pykep as pk
import numpy as np
import paseos
from paseos import ActorBuilder, SpacecraftActor
from utils import s2pix_detector, acquire_data
from paseos.utils.load_default_cfg import load_default_cfg
import asyncio
import urllib.request
import matplotlib.pyplot as plt
from matplotlib import patches

Setting logging level of `paseos` to `INFO` to reduce the verbosity of the execution.

In [ ]:
paseos.set_log_level("INFO")

# 1) - Instantiate Sentinel 2 space actors

First of all, let's create the scaffolds for **S2B**. The scaffolds are objects which are not equipped with any `communication_device` nor `power_devices`. Furthermore, they neither have any `orbit`. 

In [ ]:
# Define local actor
S2B = ActorBuilder.get_actor_scaffold(name="Sentinel2-B", actor_type=SpacecraftActor, epoch=pk.epoch(0))

#### 1.a) - Add an orbit for S2B

Since **S2B** is orbiting around Earth, let's define `earth` as `pykep.planet` object. 

In [ ]:
# Define central body
earth = pk.planet.jpl_lp("earth")

To find realistic orbits for **S2B**, we can exploit `Two Line Elements (TLEs)` (Downloaded on 27-10-2022). This would allow finding their ephemerides at time = 27-10-2022 12:00:00.
Please, refer to [Two-line_element_set](https://en.wikipedia.org/wiki/Two-line_element_set) to know more about TLEs.

In [ ]:
#Define today as pykep epoch (27-10-22)
today = pk.epoch_from_string('2022-10-27 12:00:00.000')

sentinel2B_line1 = "1 42063U 17013A   22300.18652110  .00000099  00000+0  54271-4 0  9998"
sentinel2B_line2 = "2 42063  98.5693  13.0364 0001083 104.3232 255.8080 14.30819357294601"
sentinel2B = pk.planet.tle(sentinel2B_line1, sentinel2B_line2)

#Calculating S2B ephemerides.
sentinel2B_eph=sentinel2B.eph(today)


We will now leverage the `ActorBuilder` to build a `space actor` for **S2B** on top of the scaffold previously created.

In [ ]:
#Adding orbits around Earth based on previously calculated ephemerides
ActorBuilder.set_orbit(actor=S2B, 
                       position=sentinel2B_eph[0], 
                       velocity=sentinel2B_eph[1], 
                       epoch=today, central_body=earth)

### 1.b) - Add communication and power devices

Adding power devices. Max battery level: 102 Ah @ 28V is 2.81 kWh (effect of aging is neglected).  To identify the charging rate, we are assuming to be dependent only by the power of solar panels. We are neglecting eventual charging rate limits due to battery technology. The solar-panels power is 2300 W at begin of life and 1730 W at end of life. <br> Please, refer to: [Copernicus: Sentinel-2](https://www.eoportal.org/satellite-missions/copernicus-sentinel-2#space-segment.)

In [ ]:
ActorBuilder.set_power_devices(actor=S2B, 
                               battery_level_in_Ws=10080000, 
                               max_battery_level_in_Ws=10080000, 
                               charging_rate_in_W=1860)

# 2) - Instantiate PASEOS simulation

To instantiate `PASEOS`, we consider **S2B** as `local_actor`. The initial time is set to `today`.

In [ ]:
# init simulation
cfg=load_default_cfg()
cfg.sim.start_time=today.mjd2000 * pk.DAY2SEC
cfg.sim.activity_timestep = 0.5 # update rate for plots. 
sim = paseos.init_sim(S2B, cfg)

## 3) - Dealing with PASEOS activities 

Here we demonstrate how you can use `PASEOS` to perform activities. To this aim, we will use `detection of volcanic eruptions` as use case, assuming it will be possible to run it onboard `S2B`. <br>We exploit a simplified version of the algorithm `[1]`, which leverages the spectral bands `B8A`, `B11` and `B12` to create a binary-map containing pixels marked as thermal anomalies. To detect the volcanic eruptions, we cluster those pixels in bounding boxes and return their coordinates to simulate an alert creation. <br> We will assume `S2B` will acquire images and then process them by using [1]. In this notebook, we assume to be able to detect the following images regardless of their proper geographical position. We thank the `ESA` project `Sentinel2_L0` for providing images and the implementation of [1]. <br>
**References**: <br>
`[1]  Massimetti, Francesco, et al. ""Volcanic hot-spot detection using SENTINEL-2: a comparison with MODIS–MIROVA thermal data series."" Remote Sensing 12.5 (2020): 820.`

The next cell will download the files that are needed to execute the next parts of the notebook. The files belong to the `Sentinel2_L0` dataset project (to be released soon with GPL license).

In [ ]:
if not(os.path.isfile("Etna_00.tif")):
    print("Downloading the file: Etna_00.tif")
    urllib.request.urlretrieve("https://actcloud.estec.esa.int/actcloud/index.php/s/9Tw5pEbGbVO3Ttt/download", "Etna_00.tif")
if not(os.path.isfile("La_Palma_02.tif")):
    print("Downloading the file: La_Palma_02.tif")
    urllib.request.urlretrieve("https://actcloud.estec.esa.int/actcloud/index.php/s/vtObKJOuYLgdPf4/download", "La_Palma_02.tif")
if not(os.path.isfile("Mayon_02.tif")):
    print("Downloading the file: Mayon_02.tif")
    urllib.request.urlretrieve("https://actcloud.estec.esa.int/actcloud/index.php/s/e0MyilW1plYdehL/download", "Mayon_02.tif")

### 3.a) - PASEOS visualization 

`PASEOS` offers a visualization tool that can be used to display the position of `S2B` around its orbit, the state of charge of the battery, and the name of the performed activity. <br>To enable the visualization, let's define a `plotter`. 

In [ ]:
# Plot current status of PASEOS and get a plotter
plotter = paseos.plot(sim, paseos.PlotType.SpacePlot)

### 3.b) - Registering activities 

The next lines will show how to register an activity to simulate data acquisition on **S2B**. <Br>Each activity is bound to an `asynchronous function`.  Let's define the asynchornous function `acquire_data_async()` as follow.

In [ ]:
async def acquire_data_async(args):
    args[1][0], args[2][0]=acquire_data(args[0])
    await asyncio.sleep(3.6) #Acquisition for an L0 granule takes 3.6 seconds for S2B. 

The function `acquire_data_async()` uses `args[0]` as a filename to load the file and saves the acquired image into `args[1][0]` and geo-information into `args[2][0]`. Please, notice that using `args[1][0]` instead of `args[1]` is just a trick to be sure the acquired data are modifies the input argument `args[1]` by reference without creating a local copy of it. 
Therefore, let's define a placeholder `data_acquired` as a list containing 3 lists with `None` elements, which will be overwritted by `acquire_data_async()`.

In [ ]:
data_acquired=[[None], [None], [None]]
data_acquired_coordinates=[[None], [None], [None]]

To enable the visualization of the activity in the `plotter`, we can leverage a **constraint function**. 
The purpose of a **constraint function** is generally to model constraints and return whether those are still true. However, since it is regularly executed we can use it to update the plotter  according to the simulation. Let's define `constraint_func()` as follows.

In [ ]:
# Constraint function
async def constraint_func(args):
    plotter.update(sim)
    return True #Do not remove this.

The next line will register the activity `data_acquisition` by associating int to the `activity_function` previously defined, `acquire_data_async` and by using the `constraint_func` as **constraint function**. It is up to the user to specify the power consumption in W for the activity. For Sentinel-2, the peak power of the imager is 266 W as reported in [Copernicus: Sentinel-2](https://www.eoportal.org/satellite-missions/copernicus-sentinel-2#space-segment.)

In [ ]:
# Register an activity that emulate data acquisition
sim.register_activity(
    "data_acquisition", activity_function=acquire_data_async, 
    power_consumption_in_watt=266, constraint_function=constraint_func
)

Similary, we can register an activity to perform the volcanic event detection. 

In [ ]:
# Output bbox info
output_event_bbox_info = [[None], [None], [None]]

async def detect_volcanic_eruptions_async(args):
    args[2][0] = s2pix_detector(args[0][0], args[1][0])
    await asyncio.sleep(1) #Assuming one second processing for the cropped tile.


# Register an activity that emulate event detection
sim.register_activity(
    "volcanic_event_detection",
    activity_function=detect_volcanic_eruptions_async,
    power_consumption_in_watt=10,
    constraint_function=constraint_func
)

Let's define now a new activity emulating the status in which the satellite is an await status before to be able to collect new images. 

The new activity `wait_before_acquire_async(time_s)` is used to simulate a status where the satellite is neither acquiring data, neither processing data. `time_s` is the wait time. The function will be also useful to showcase the `PASEOS` visualization utilities for long time. To this aim, you can set `time_s` beyond 10s to extend the visualization plotting above. <br>

In [ ]:
async def wait_before_acquire_async(time_s):
    print("Waiting to be able to acquire new data.")
    await asyncio.sleep(time_s[0])

We also use the function `on_termination`, which is executed once the activity is terminated. In this example, this is a signal that notifies that the satellite is ready to start acquiring again. 

In [ ]:
async def on_termination(args):
    print("Satellite ready to acquire data again.")

Let's now register the activity `wait_before_acquire` by using both `constraint_func` and `on_termination` functions. We set up a not-zero `power_consumption_in_watt` for the activity to show you how the battery status is updated in the visualization. 

In [ ]:
# Register an activity that emulate event detection
sim.register_activity(
    "wait_before_acquire", 
    activity_function=wait_before_acquire_async, 
    power_consumption_in_watt=20000, 
    constraint_function=constraint_func,
    on_termination_function=on_termination
)

### 3.c) - Performing activities.

We can now perform the activities. In scheduling the activities, we assume that data are acquired and stored into the mass memory to be, then, processed during the off part of the satellite duty cycle. Please, refer to: [Copernicus: Sentinel-2](https://www.eoportal.org/satellite-missions/copernicus-sentinel-2#space-segment.). <br> **N.B.** Notice that, when `sim.perform_activity(...)` is called, the keyword `await` is needed  **only for Jupyter notebooks** to ensure the correct management of asynchronous functions. **Remove it otherwise**. 

In [ ]:
data_to_acquire=["Etna_00.tif", "La_Palma_02.tif", "Mayon_02.tif"]


#Data acquisition and processing
print("Scroll up and take a look at the plotter.")

for n, data_name in zip(range(len(data_to_acquire)), data_to_acquire):
    # Run the activity
    await sim.perform_activity("wait_before_acquire", activity_func_args=[10])
    
    #Run the activity
    await sim.perform_activity("data_acquisition", activity_func_args=[data_name, data_acquired[n], data_acquired_coordinates[n]])
    
    #Run the activity
    await sim.perform_activity("volcanic_event_detection", 
                               activity_func_args=[data_acquired[n], data_acquired_coordinates[n], output_event_bbox_info[n]])

    
# Updating the plotter outside to show the final state after performing the activities
plotter.update(sim)


### 3.d) - Showing detected volcanic eruptions

The next plot will show an example of onboard coarse volcanic eruptions detection on some Sentinel-2 L1C tiles. The different eruptions will be surrounded a bounding box, and their coordinates will be printed to raise an alert.

In [ ]:
bboxes, bbox_coordinates = output_event_bbox_info[n][0][0], output_event_bbox_info[n][0][1]

In [ ]:
fig, ax=plt.subplots(1,3,figsize=(10,4))

for n in range(3):
    ax[n].imshow(data_acquired[n][0])
    
    bboxes, bbox_coordinates = output_event_bbox_info[n][0][0], output_event_bbox_info[n][0][1]
    for bbox in bboxes:
        bbox=bbox.bbox
        rect = patches.Rectangle((bbox[1], bbox[0]), abs(bbox[1]-bbox[3]), abs(bbox[0]-bbox[2]), linewidth=1, edgecolor='y', facecolor='none')
        ax[n].add_patch(rect)
    
    for coords in bbox_coordinates:
        print("ALERT! Eruption found at: \n\t Top left corner(lon, lat): "+str(coords[0]) +"\n\t bottom right corner(lon, lat): "+str(coords[1])+"\n")
    print("\n")